# Setup

In [1]:
# my virtual environments are rarely properly connected to jupyter so this fixes that. 
# you prolly won't need this cell but running it won't hurt anything either
import sys
import os
current_dir = os.getcwd()  # Get the current working directory
venv_dir = os.path.join(current_dir, 'venv') 
python_version = str(sys.version_info.major) + '.' + str(sys.version_info.minor)
site_packages_path = os.path.join(venv_dir, 'lib', 'python' + python_version, 'site-packages')
sys.path.append(site_packages_path) 

# Instantiate a brand new model

In [2]:
# config file
from config import ModelConfig, TrainConfig
cfg = ModelConfig()
tcfg = TrainConfig()
print(cfg, '\n\n', tcfg)

# import the tokenizer specified by cfg
from tools import import_from_nested_path
imported_objects = import_from_nested_path(['tokenizers', cfg.tokenizer], 'tokenizer', ['get_tokenizer'])
get_tokenizer = imported_objects.get('get_tokenizer')
tokenizer = get_tokenizer(size = cfg.vocab_len)

# model modules
from modules.model import Model
model = Model(cfg).to(cfg.device)

# print the number of parameters in the model
print(f'\n{sum(p.numel() for p in model.parameters())/1e3}K parameters\n')
print(model)

ModelConfig(dim=80, device='cpu', tokenizer='bpe_v1', vocab_len=8192, num_layers=4, second_resid_norm=False, num_heads=2, head_dim=40, max_seq_len=512, mm_bias=False, pmem_size=224, pmem_count=1, scale_first_resid=True, norm_type='RMSNorm', norm_affine=True, norm_bias=True, eps=1e-06) 

 TrainConfig(weight_decay=0.05, batch_size=32, max_iters=2000, eval_interval=20, eval_samples=1, checkpoint_interval=None, lr_init=1e-06, lr_max=0.1, lr_min=0.001, warmup_iters=20, final_flat_iters=400, anneal_type='cos', num_restarts=3, T_mult=2)

928.456K parameters

Model(
  (token_embedder): Embedding(8195, 80)
  (layers): ModuleList(
    (0-3): 4 x Layer(
      (pre_context_norm): Norm()
      (context): ContextMem(
        (k_featurizer): KeyFeatureExtractor(
          (W_k): Linear(in_features=80, out_features=80, bias=False)
          (leaky_avg): LeakyAvg()
        )
        (v_featurizer): ValFeatureExtractor(
          (W_v): Linear(in_features=80, out_features=80, bias=False)
        )
     

# Training

In [3]:
import torch
from tools import get_data_loader
from train import scheduler_lambda, train

optimizer = torch.optim.AdamW(model.parameters(), lr = tcfg.lr_max, weight_decay = tcfg.weight_decay)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=scheduler_lambda)

train_data_loader = get_data_loader(batch_size=tcfg.batch_size, split='train')
test_data_loader = get_data_loader(batch_size=tcfg.batch_size, split='validation')

In [4]:
if False: # set to true if you'd like to see a graph of the learning rate schedule
    import matplotlib.pyplot as plt
    
    # Generate learning rate values
    lrs = [scheduler_lambda(i) for i in range(tcfg.max_iters)]
    
    # Plot the learning rates
    plt.figure(figsize=(10, 5))
    plt.plot(lrs, label='Learning Rate')
    plt.title('Learning Rate Schedule')
    plt.xlabel('Iteration')
    plt.ylabel('Learning Rate')
    plt.grid(True)
    plt.legend()
    plt.show()

In [5]:
model, optimizer, log_data = train(
    model, 
    tokenizer, 
    cfg, 
    optimizer,
    scheduler,
    tcfg, 
    train_data_loader,
    test_data_loader,
    #log_data: list = None, 
    #detect_anomoly = False # use if you're getting crazy errors about a the gradient being broken
)

  0%|                                                                         | 1/2000 [00:19<10:38:45, 19.17s/it]

step 0000: lr 0.000000, train loss 77.6600, val loss 77.4064, ppl 4141650957493128962322235104690176, time elapsed: 15.92 seconds


  1%|▊                                                                        | 21/2000 [04:18<3:10:40,  5.78s/it]

step 0020: lr 0.010000, train loss 29.7979, val loss 29.8511, ppl 9208346968064, time elapsed: 255.80 seconds


  2%|█▍                                                                       | 41/2000 [06:15<3:41:47,  6.79s/it]

...

step 1960: lr 0.000100, train loss 1.7415, val loss 1.7295, ppl 6, time elapsed: 7923.38 seconds


 99%|██████████████████████████████████████████████████████████████████████▎| 1981/2000 [2:13:09<01:11,  3.79s/it]

step 1980: lr 0.000100, train loss 1.8125, val loss 1.7438, ppl 6, time elapsed: 7987.13 seconds


100%|███████████████████████████████████████████████████████████████████████| 2000/2000 [2:14:10<00:00,  4.03s/it]

step 1999: lr 0.000100, train loss 1.7581, val loss 1.7197, ppl 6, time elapsed: 8048.32 seconds


# inference test before you decide to save it

In [6]:
from inference import generate
prompt = "Once upon a time"
model.eval()
output = generate(
    prompt, 
    model, 
    tokenizer,
    #max_gen_len = 512,
    temperature = 0.9,
    #memory_saver_div = 8,
    #top_p = 0.9,
    #top_k = 32,
)
model.train()
print(output)

Once upon a time, there was a chake it was sad and special chake the chake it was a chake it saw a big chake. The chake the chake it did not real flow and found a girl and played back to play with his bird was happy. The chake it kept magirl and playing that the chake the chake the chake the chake the chake the chake. The chake the chake the chake the chake it and played to be the chake and flew the chake. The chake the chake the chake of the chake. The chake the chake the chake the chake the chake. The chake the chake a chake the chake the chake the chake it goo was a pretty. The chake the chake the chake the chake to playing together of the chake the chake the chake for her came and flew the chake the chake. The chake the chake the chake the chake the chake. The chake it


# Saving your final model
if `tcfg.checkpoint_interval != None` then checkpoints have already been saved

you DO still need to do this even if you had been saving checkpoints; the final state has not yet been saved

In [7]:
from tools import save_model
save_model(model, cfg, tcfg, log_data)